## **Introduction**

Nama    : Intan Mukti Pebriana<br>
Batch   : HCK 019<br>
Objective   : Melakukan prediction rating pada model inference setelah membuat model NLP yaitu, analisis ulasan pengguna dari aplikasi Livin' by Bank Mandiri.

---

## **Process**

## **Import Libraries**

In [2]:
# !pip install pysastrawi
# !pip install tensorflow==2.15

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled

In [21]:
# import streamlit as st
import pandas as pd
import re
import tensorflow as tf
import tensorflow_hub as tf_hub
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import nltk

In [22]:
# Download the stopwords resource
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## **Load Data Inference**

In [23]:
# Menampilkan dataset menjadi dataframe
X = pd.read_csv("X_inf.csv")
X

,review
0,Pokoknya mantap deh
1,Good
2,Bertahan lama bgt gak mau update kesini. Akhir...
3,Kenapa gak bisa d buka aplikasi Livin nya..tib...
4,Terbaik💥💥
5,cukup baik👍
6,Transaksi semudah menjetikan jari.
7,TOLONG JANGAN TERLALU SERING UPDATE YAAAAH!!!!...
8,Mudah menggunakan aplikasi ini dan informasi s...
9,Susah transaksi


In [24]:
# Menampilkan dataset menjadi dataframe
y = pd.read_csv("y_inf.csv")
y

,rating
0,5
1,5
2,1
3,2
4,5
5,4
6,5
7,3
8,5
9,1


## **Text Preprocessing**

In [25]:
# Define Stopwords
from nltk.corpus import stopwords
stpwds_id = list(set(stopwords.words('indonesian')))
# stpwds_id.append('oh')
# Daftar kata yang ingin dicek
kata = ['livin', 'update', 'aplikasi', 'mandiri', 'transaksi', 'buka', 'biru', 'gunta', 'brp','hr','donlon',
        'kuning', 'bank', 'banget', 'bilang', 'tolong', 'knapa','skrg', 'bs', 'dmana', 'bgt', 'donlod','dg', 'taun',
        'mohon', 'kali', 'pakai', 'rekening', 'logo', 'kartu', 'coba', 'knp', 'lot', 'ko','app', 'wrn','pke',
        'pulsa', 'pake', 'suruh', 'daftar', 'kasih', 'bikin', 'lot', 'ganti', 'nih', 'deh','jg','tf','thn','nope',
        'lebih', 'lbh', 'jd', 'dgn', 'jk', 'depa', 'tampi','bdanya','admin','tmen','am','kryawan','bolak','hrs',
        'gimana', 'mulu', 'kalo', 'bintang', 'menu', 'hasil', 'pasword', 'download', 'trus', 'stlh','td','cm',
        'instal', 'uang', 'moga', 'habis', 'butuh', 'upgrade', 'data', 'dah', 'sdh', 'dr', 'msh', 'lg','hny',
        'dana', 'nomor', 'potong', 'langsung', 'shopee', 'tpy',  'klo', 'apk', 'sih', 'sy', 'by','skr','trs',
        'online', 'banking', 'sampe', 'gin', 'shopeepay', 'karna', 'kaya', 'biar', 'prnh','bla','sts', 'isteri',
        'moncer', 'donk', 'perban', 'ap', 'udh', 'tp', 'knpa', 'living', 'malam', 'pindah', 'diupdate', 'email',
        'gw', 'endap', 'ny', 'cman', 'rb', 'bln', 'skg', 'sbelumnya', 'ngsih', 'rgular', 'blan', 'risen', 'lho',
        'uninstal', 'pilih', 'yg', 'nya', 'ya', 'aja', 'udah','utk','atm','tntg','dn','hadeh','jmn','jrg','org','pnya']

# Tambahkan kata-kata tersebut ke dalam daftar stopwords
stpwds_id_add = stpwds_id + kata

# Define Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stemmer = StemmerFactory().create_stemmer()

In [26]:
def text_preprocessing(text):
    # Mengubah teks menjadi huruf kecil (case folding)
    text = text.lower()

    # Kamus pengganti kata slang
    slang_dict = {
        # Kata yang terkait dengan "tidak"
        "ga": "tidak", "gak": "tidak", "gk": "tidak", "ngk": "tidak", "ngak": "tidak",
        "engak": "tidak", "engga": "tidak", "ngga": "tidak", "nggak": "tidak",
        "gx": "tidak", "enggak": "tidak", "tdk": "tidak", 'nggk': 'tidak',

        # Kata yang terkait dengan "upload"
        "uplod": "upload", "ngaplod": "upload", "aplod": "upload", "nguplod": "upload",

        # Kata yang terkait dengan "masalah"
        "mslh": "masalah"
    }

    # Mengganti kata slang menggunakan kamus yang sudah dibuat
    pattern = re.compile(r'\b(' + '|'.join(slang_dict.keys()) + r')\b')
    text = pattern.sub(lambda x: slang_dict[x.group()], text)

    # Menghapus kata yang diakhiri dengan 'x' (misal, akhirx -> akhir, biasax -> biasa)
    text = re.sub(r'x\b', '', text)

    # Menghapus kata yang dimulai dengan 'br' (misal, brulang -> ulang)
    text = re.sub(r'\bbr([a-z]+)', r'\1', text)

    # Menghapus karakter non-huruf (hanya menyimpan alfabet dan spasi)
    text = ''.join([char if char.isalpha() or char.isspace() else ' ' for char in text])

    # Menghapus spasi berlebihan (menjaga hanya satu spasi antar kata)
    text = ' '.join(text.split())

    # Menghapus huruf vokal yang berulang lebih dari 3 kali berturut-turut (misal, 'baguuus' -> 'bagus')
    text = re.sub(r'([aeiou])\1{2,}', r'\1', text)

    # Menghapus huruf konsonan yang berulang lebih dari sekali berturut-turut (misal, 'kerenn' -> 'keren')
    text = re.sub(r'([^aeiougrc])\1+', r'\1', text)

    # Tokenisasi (memecah teks menjadi kata-kata terpisah)
    tokens = word_tokenize(text)

    # Menghapus kata yang hanya terdiri dari 1 karakter
    tokens = [word for word in tokens if len(word) > 1]

    # Menghapus stopwords (dengan asumsi 'stpwds_id_add' sudah didefinisikan sebelumnya)
    tokens = [word for word in tokens if word not in stpwds_id_add]

    # Proses stemming (dengan asumsi 'stemmer' sudah didefinisikan sebelumnya)
    tokens = [stemmer.stem(word) for word in tokens]

    # Menggabungkan kembali token menjadi satu string
    text = ' '.join(tokens)

    return text


In [27]:
X_resample = X['review'].apply(text_preprocessing)
X_resample

,review
0,pokok mantap
1,good
2,tahan kesini paksa kesini tau parah parah jele...
3,error
4,baik
5,
6,mudah menjetikan jari
7,yah
8,mudah informasi lengkap
9,susah


# **Predict**

In [28]:
# Load the trained model
model = load_model('model_lstm_2.keras')

In [36]:
import numpy as np

# Menghasilkan prediksi dari model
predict = model.predict(X_resample)

# Mengakses probabilitas untuk kelas dengan probabilitas tertinggi
probabilities = predict[0]  # Asumsi model mengeluarkan array 2D dan kita ambil array 1D dari prediksi
max_probability = np.max(probabilities)  # Ambil nilai probabilitas tertinggi
rating_class = np.argmax(probabilities) + 1  # Kelas rating berdasarkan probabilitas tertinggi

# Menentukan rating berdasarkan probabilitas
if max_probability > 0.6:
    rating = 5
elif max_probability > 0.4:
    rating = 4
elif max_probability > 0.3:
    rating = 3
elif max_probability > 0.2:
    rating = 2
else:
    rating = 1

print(f"Predicted rating: {rating}")


1/1 [==============================] - 1s 877ms/step
Predicted rating: 1


In [37]:
import pandas as pd
import numpy as np


# Menghitung rating berdasarkan probabilitas
ratings = []
for prob in predict:
    max_probability = np.max(prob)
    if max_probability > 0.6:
        rating = 5
    elif max_probability > 0.4:
        rating = 4
    elif max_probability > 0.3:
        rating = 3
    elif max_probability > 0.2:
        rating = 2
    else:
        rating = 1
    ratings.append(rating)

# Membuat DataFrame dengan rating
df = pd.DataFrame({
    'Prediction': list(predict),
    'Rating': ratings
})

# Menampilkan DataFrame
print(df)


       Prediction  Rating
0    [0.19029942]       1
1     [0.7353552]       5
2  [0.0001490312]       1
3    [0.38938612]       3
4    [0.38938612]       3
5      [0.658649]       5
6    [0.08862169]       1
7    [0.38938612]       3
8    [0.08862169]       1
9    [0.38938612]       3


In [38]:
df['review'] = X_resample

In [39]:
df

,Prediction,Rating,review
0,[0.19029942],1,pokok mantap
1,[0.7353552],5,good
2,[0.0001490312],1,tahan kesini paksa kesini tau parah parah jele...
3,[0.38938612],3,error
4,[0.38938612],3,baik
5,[0.658649],5,
6,[0.08862169],1,mudah menjetikan jari
7,[0.38938612],3,yah
8,[0.08862169],1,mudah informasi lengkap
9,[0.38938612],3,susah
